# Tutorial on how to use mescal
*mescal* is a [Brightway](https://docs.brightway.dev/en/latest/)-powered Python package that helps you to integrate Life-Cycle Assessment (LCA) in your energy system model

## Set up

In [1]:
# Import the required libraries
from mescal import ESM, Database
import pandas as pd
import bw2data as bd

In [2]:
ecoinvent_version = '3.9.1' # choose the ecoinvent version you wish to use
esm_location = 'CA-QC' # choose the version of energyscope for which you want to generate metrics
esm_year = 2020 # choose the year of the energyscope snapshot model or transition pathway time-step
spatialized_database = True # set to True if you want to use your spatialized version of ecoinvent
regionalize_foregrounds = True # set to True if you want to regionalize the foreground inventories
premise_iam = 'image' # choose the IAM to which the premise database is linked
premise_ssp_rcp = 'SSP2-Base' # choose the SSP/RCP scenario to which the premise database is linked

In [3]:
# Set the name of your main LCI database (e.g., ecoinvent or premise database) here:
name_premise_db = f"ecoinvent_cutoff_{ecoinvent_version}_{premise_iam}_{premise_ssp_rcp}_{esm_year}+truck_carculator"
if spatialized_database:
    name_premise_db += ' regionalized'
    name_spatialized_biosphere_db = 'biosphere3_spatialized_flows'
else:
    name_spatialized_biosphere_db = None

In [4]:
# Set the name of the new database with the ESM results
esm_db_name = f'EnergyScope_{esm_location}_{esm_year}'

In [5]:
# Set the list of LCIA methods for which you want indicators (they must be registered in your brightway project)
if spatialized_database:
    lcia_methods=[
        'IMPACT World+ Midpoint 2.0.1_regionalized', 
        'IMPACT World+ Damage 2.0.1_regionalized', 
        'IMPACT World+ Footprint 2.0.1_regionalized', # IW+ regionalized
    ]
else:
    lcia_methods=[
        'IMPACT World+ Midpoint 2.0.1', 
        'IMPACT World+ Damage 2.0.1', 
        'IMPACT World+ Footprint 2.0.1', # IW+
        'PB LCIA' # Planetary Boundaries
    ]

**Note**: you can download IMPACT World+ methods in your brightway project following this [notebook](https://github.com/matthieu-str/mescal/blob/master/dev/download_impact_world_plus.ipynb). Regionalized versions of ecoinvent and IMPACT World+ methods can be downloaded from this [repository](https://github.com/CIRAIG/Regioinvent).

In [6]:
# Set up your Brightway project
bd.projects.set_current(f'ecoinvent{ecoinvent_version}') # put the name of your brightway project here

## Load databases

In [7]:
premise_db = Database(name_premise_db, create_pickle=True)

Loaded ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+truck_carculator regionalized from pickle!


In [8]:
if regionalize_foregrounds & spatialized_database:
    spatialized_biosphere_db = Database(name_spatialized_biosphere_db)
else:
    spatialized_biosphere_db = None

Getting activity data


100%|██████████| 62222/62222 [00:00<00:00, 72593.14it/s] 


Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


100%|██████████| 62222/62222 [00:00<00:00, 4274263.54it/s]


Loaded biosphere3_spatialized_flows from brightway!


## Your data
For mescal to understand the structure of your energy system model, you need to provide it with a set of dataframes.

### Mandatory dataframes

In [9]:
mapping = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}.csv')
unit_conversion = pd.read_csv(f'../dev/energyscope_data/{esm_location}/unit_conversion_{ecoinvent_version}.csv')
mapping_esm_flows_to_CPC = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_esm_flows_to_CPC.csv')
model = pd.read_csv(f'../dev/energyscope_data/{esm_location}/model.csv')

A mapping between the energy technologies and resources of the energy system model, and Life-Cycle Inventories datasets (LCI) from an LCI database (e.g., ecoinvent). The mapping should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology or resource in the energy model 
- `Type`: the type of the energy technology or resource (i.e., 'Construction', 'Operation', 'Resource' or 'Flow')  
- `Product`: the name of the product of the energy technology or resource in the LCI database
- `Activity`: the name of the activity of the energy technology or resource in the LCI database
- `Location`: the name of the location of the energy technology or resource in the LCI database
- `Unit`: (optional) the physical unit of the energy technology or resource in the LCI database
- `Database`: the name of the database in your brightway project

If you wish to change the version of ecoinvent used in your mapping file, you can follow this [notebook](https://github.com/matthieu-str/mescal/blob/master/dev/mapping_ecoinvent_version.ipynb). 

In [10]:
mapping.head()

,Name,Type,Product,Activity,Location,Database
0,AFC,Construction,"fuel cell system, 1 kWe, proton exchange membr...","fuel cell system assembly, 1 kWe, proton excha...",GLO,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020
1,AFC,Operation,"electricity, from residential heating system","electricity, residential, by conversion of hyd...",CH,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020
2,ALKALINE_ELECTROLYSIS,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020
3,ALKALINE_ELECTROLYSIS_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020
4,ALKALINE_ELECTROLYSIS_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020


A set of unit conversion factors between the energy system model and the LCI database. The conversion factors should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology or resource in the energy system model 
- `Type`: the type of the energy technology or resource (i.e., 'Construction', 'Operation', 'Resource', or 'Other'). Other types can be added to fit your needs. The 'Other' category is meant for the unit conversion that are not specific to a technology or resource, but rather a generic type of product, e.g., conversion from kilogram to kilowatt hour for natural gas.
- `Value`: the numerical value of the conversion factor that will multiply the impact scores. It actually denotes the conversion from Impact / LCA unit to Impact / ESM unit. 
- `ESM`: the unit of the energy technology or resource in the ESM
- `LCA`: the target unit of the energy technology or resource in the LCA database

The `LCA` and `ESM` columns should respect the ecoinvent naming convention. You may use the `ecoinvent_unit_convention` function to convert the unit naming convention you've used to the one of ecoinvent.

In [11]:
unit_conversion.head()

,Name,Type,Value,LCA,ESM
0,ACETIC_ACID,Resource,0.247423,kilogram,kilowatt hour
1,ACETONE,Resource,0.121655,kilogram,kilowatt hour
2,AFC,Construction,1.000000,unit,kilowatt
3,AFC,Operation,1.000000,kilowatt hour,kilowatt hour
4,ALKALINE_ELECTROLYSIS,Construction,0.001556,unit,kilowatt


A mapping between the energy system model flows and CPC categories. The mapping should be provided in a dataframe with the following columns:
- `Flow`: the name of the flow in the ESM
- `Description`: (optional) the description of the product
- `CPC`: the list of names of corresponding CPC categories

In [12]:
mapping_esm_flows_to_CPC.head()

,Flow,Description,CPC
0,BENZENE,Benzene,"['33100: Coke and semi-coke of coal, of lignit..."
1,BIO_DIESEL,Bio-diesel,"['35491: Biodiesel', '33370: Fuel oils n.e.c.'..."
2,CO2_A,Carbon dioxide (concentrated emissions),['34210b: Carbon dioxide and monoxide']
3,CO2_C,Carbon dioxide (captured),['34210b: Carbon dioxide and monoxide']
4,CO2_CS,Carbon dioxide (captured & stored),['34210b: Carbon dioxide and monoxide']


The input and output flows of energy technologies. For a given technology, the inputs and outputs should be given with the same unit. Also, the main output flow should have 1 as an amount, i.e., all other flows as scaled with respect to the main output. It should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy system model
- `Flow`: the name of the input or output flow
- `Amount`: the numerical value of the flow (negative if input, positive if output)  

In [13]:
model.head()

,Name,Flow,Amount
0,DIESEL_S,DIESEL_S,1.0
1,GASOLINE_S,GASOLINE_S,1.0
2,ELEC_S,ELEC_S,1.0
3,NG_S,NG_S,1.0
4,ELECTRICITY_EHV,ELECTRICITY_EHV,1.0


### Optional dataframes

In [14]:
technology_compositions = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_compositions.csv')
technology_specifics = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_specifics.csv') 
lifetime = pd.read_csv(f'../dev/energyscope_data/{esm_location}/lifetime.csv')
efficiency = pd.read_csv(f'../dev/energyscope_data/{esm_location}/efficiency.csv')
mapping_product_to_CPC = pd.read_csv('../mescal/data/mapping_product_to_CPC.csv')
impact_abbrev = pd.read_csv('../dev/lcia/impact_abbrev.csv')
technologies_to_remove_from_layers = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technologies_to_remove_from_layers.csv')
new_end_use_types = pd.read_csv(f'../dev/energyscope_data/{esm_location}/new_end_use_types.csv')
premise_changes = pd.read_csv('../dev/data/premise_change_report.csv')
results_from_esm = pd.read_csv(f'../dev/energyscope_data/{esm_location}/results_ES.csv')

A set of composition of technologies, i.e., if one technology or resource in the energy model should be represented by a combination of LCI datasets. The composition should be provided in a dataframe with the following columns:
- `Name`: the name of the main energy technology or resource in the energy model 
- `Components`: the list of names of subcomponents

In [15]:
technology_compositions.head()

,Name,Components
0,ALKALINE_ELECTROLYSIS,"['ALKALINE_ELECTROLYSIS_STACK', 'ALKALINE_ELEC..."
1,AN_DIG_SI,"['AN_DIG_SI_PLANT', 'AN_DIG_SI_COGEN']"
2,ATR,"['ATR_PLANT', 'ATR_TANK']"
3,ATR_CCS,"['ATR_CCS_PLANT', 'ATR_CCS_TANK']"
4,BIOGAS_ATR,"['BIOGAS_ATR_PLANT', 'BIOGAS_ATR_TANK']"


A set of technologies with specific requirements. For instance, this stands for energy technologies without a construction phase, mobility technologies (if mismatch fuel in the LCI dataset), bio-processes (if mismatch fuel in the LCI dataset), etc. The requirements should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy model
- `Specifics`: the type of requirement. Can be '**Background search**' (i.e., double-counting removal is run n levels in the background, n being defined in Amount), '**Decommissioning**' (i.e., the technology has a decommissioning LCI dataset outside its infrastructure LCI dataset), '**Mobility**' (i.e., EUD types for which associated technologies are characterized as a mobility mean, to further add a FUEL layer), '**No background search**' (i.e., double-counting removal is not applied), '**No construction**' (i.e., the technology has no infrastructure LCI dataset), '**Process**' (i.e., the technology is characterized as an industrial bio-process, to further add a PROCESS_FUEL layer), '**DAC**' (for *premise* DAC technologies, to change the biogenic carbon flow into a fossil one), '**Biofuel**' (i.e., adapt direct emissions to the biofuel input: partially change fossil carbon emissions into biogenic ones), '**Import/Export**' (these activities will keep their original locations and will not be regionalized), '**Carbon flows**' (the direct carbon emissions of these activities will be multiplied by a factor). 
- `Amount`: the numerical value of the requirement (if relevant)

In [16]:
technology_specifics.head()

,Name,Specifics,Amount,Comment
0,CO2_TO_DIESEL,Background search,2.0,NaN
1,CO2_TO_JETFUELS,Background search,0.0,NaN
2,CROPS_TO_JETFUELS,Background search,4.0,NaN
3,FT,Background search,3.0,NaN
4,GASIFICATION_SNG,Background search,2.0,NaN


Energy technologies lifetimes in the ESM and the LCI database. For composition of technologies (if any), the main technology should not have a LCA lifetime (no LCI dataset is associated to it), while the sub-components should not have an ESM lifetime (they do not have a proper technology in the ESM). The lifetimes should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy system model
- `ESM`: the numerical value of the lifetime in the energy system model (if relevant)
- `LCA`: the numerical value of the lifetime in the LCI database (if relevant)
- `Comment`: (optional) additional information the `Amount` computation (if relevant)

In [17]:
lifetime.head()

,Name,ESM,LCA
0,AFC,35.0,2.686006
1,AL_MAKING,25.0,50.000000
2,AL_MAKING_HR,25.0,50.000000
3,ALKALINE_ELECTROLYSIS,10.0,NaN
4,ALKALINE_ELECTROLYSIS_PLANT,NaN,20.000000


If you want to account for the possible efficiency differences between the ESM and the LCI datasets, you can provide a set of (technologies, flow) couples for which efficiency differences will be corrected by the scaling the elementary flows. Taking the example of the diesel car, the couple should be something like ('CAR_DIESEL', 'DIESEL'), the flow being the fuel of the technology responsible for direct emissions. Relevant biosphere flows will be the ratio between the LCA and ESM efficiencies. The (technology, flow) couples should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the ESM
- `Flow`: the name of the flow in the ESM
- `Comment`: (optional) additional information on the set of `Flows`

In [18]:
efficiency.head()

,Name,Flow,Comment
0,AN_DIG,['WET_BIOMASS'],NaN
1,AN_DIG_SI,['WET_BIOMASS'],NaN
2,ATR,['NG_HP'],NaN
3,ATR_CCS,['NG_HP'],NaN
4,BIOGAS_ATR,['WOOD'],NaN


In case you have a LCI database (partially) without CPC categories (which are necessary for the double-counting check), you can provide a mapping between the products and activities in the LCI database and the CPC categories. The mapping should be provided in a dataframe with the following columns:
- `Name`: the full or partial name of the product or activity in the LCI database
- `CPC`: the number and name of the corresponding CPC category
- `Search type`: can be 'equals' if the `Name` entry is an exactly the name to look for, or 'contains' if it is contained in the full name
- `Where`: can be 'Product' or 'Activity', whether the `Name` entry is meant for products or activities

In [19]:
mapping_product_to_CPC.head()

,Name,CPC,Search type,Where
0,amine-based silica,"35310: Organic surface active agents, except soap",equals,Product
1,biodiesel,35491: Biodiesel,contains,Product
2,biogas,"17200: Coal gas, water gas, producer gas and s...",equals,Product
3,"biomass, used as fuel",31230: Wood in chips or particles,equals,Product
4,"biomethane, from biogas upgrading, using amine...","12020: Natural gas, liquefied or in the gaseou...",equals,Product


An abbreviation scheme for the impact categories you aim to work with, to ease the readability in the ESM. The one of IMPACT World+ is available in this [csv file](https://github.com/matthieu-str/mescal/blob/master/dev/lcia/impact_abbrev.csv). The abbreviations should be provided in a dataframe with the following columns:
- `Impact_category`: the name of the impact category, expressed as a tuple following brightway convention
- `Unit`: (optional) the unit of the impact category
- `Abbrev`: the abbreviation of the impact category
- `AoP`: the area of protection of the impact category. The normalization of indicators will be performed based of AoPs. If you do not have AoPs (e.g., midpoint-level indicators), set as many AoPs as you have impact categories.

In [20]:
impact_abbrev.head()

,Impact_category,Unit,Abbrev,AoP
0,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQL,EQ
1,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQS,EQ
2,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQLB,EQ
3,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQSB,EQ
4,"('IMPACT World+ Damage 2.0.1', 'Human health',...",DALY,CCHHL,HH


In case you want to remove some energy technologies from energy layers in the results LCI datasets to be added to the LCI database, you can provide a list of technologies to remove. The list should be provided in a dataframe with the following columns:
- `Layers`: name of the layer(s). A layer is basically an energy vector, which is an output for some energy technologies and an input for some others. 
- `Technologies`: the name of the energy technology to remove from the layer(s)
- `Comment`: (optional) a comment on the removal

In [21]:
technologies_to_remove_from_layers.head()

,Layers,Technologies,Comment
0,"['ELECTRICITY_EHV', 'ELECTRICITY_HV']","['TRAFO_HE','TRAFO_EH']",The high and extra high voltage electricity ar...
1,['ELECTRICITY_LV'],['STO_ELEC'],The storage technologies should be removed (pr...
2,"['NG_HP', 'NG_EHP']","['NG_EXP_EH', 'NG_EXP_EH_COGEN', 'NG_COMP_HE',...",The high and extra high pressure natural gas a...
3,"['H2_LP', 'H2_MP', 'H2_HP', 'H2_EHP']","['H2_COMP_HE', 'H2_COMP_MH', 'H2_COMP_LM', 'H2...",All pressure levels for hydrogen are merged in...
4,"['HEAT_HIGH_T', 'HEAT_LOW_T_DHN']",['HT_LT'],The high and low heat production at the DHN le...


If you want to reformat your end-use types (e.g., output layer) in order to better fit the LCI database, you can provide a list of new end-use types. The list should be provided in a dataframe with the following columns:
- `Name`: name of technologies for which the end-use type should be changed
- `Search type`: whether the `Name` entry is an exactly the name to look for ('equals'), is contained in the full name ('contains'), or is the beginning of the full name ('startswith')
- `Old`: the current end-use type
- `New`: the new end-use type

In [22]:
new_end_use_types.head()

,Name,Search type,Old,New
0,BUS,startswith,MOB_PUBLIC,MOB_PUBLIC_BUS
1,SCHOOLBUS,startswith,MOB_PUBLIC,MOB_PUBLIC_SCHOOLBUS
2,COACH,startswith,MOB_PUBLIC,MOB_PUBLIC_COACH
3,TRAIN,startswith,MOB_PUBLIC,MOB_PUBLIC_TRAIN
4,CAR,startswith,MOB_PRIVATE,MOB_PRIVATE_CAR


If you are facing changes in names or reference products due to a new version of premise, and do not want to modify your mapping file, you should provide the dataframe with the following columns:
- `Product - old`: old reference product of the activity
- `Activity - old`: old name of the activity
- `Location - old`: old location of the activity
- `Product - new`: new reference product of the activity
- `Activity - new`: new name of the activity
- `Location - new`: new location of the activity

In [23]:
premise_changes.head()

,Product - old,Activity - old,Location - old,Product - new,Activity - new,Location - new
0,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming of...",RER,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming",RER
1,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming of...",RER,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming, w...",RER
2,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming, f...",CH,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming, f...",RER
3,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming, f...",CH,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming, f...",RER
4,"hydrogen, gaseous, 26 bar","hydrogen production, steam methane reforming, ...",CH,"hydrogen, gaseous, low pressure","hydrogen production, steam methane reforming, ...",RER


If you want to inject the results of your ESM back in the LCI database, you should provide the results in a dataframe with the following columns:
- `Name`: the name of the energy technology in the ESM
- `Production`: the annual production value of the energy technology in the ESM. All values should be provided with the same unit.

In [24]:
results_from_esm.head()

,Name,Production
0,PV,0.000000
1,TRAFO_ML,72698.861206
2,AN_DIG_SI,0.000000
3,GEOTHERMAL,0.000000
4,HYDRO_GAS_CHP,0.000000


### Optional geographical data

In [25]:
# sufficient match within ecoinvent
if esm_location == 'CA-QC':
    accepted_locations = ['CA-QC', 'CA']  
elif esm_location == 'CH':
    accepted_locations = ['CH']
else:
    accepted_locations = ['GLO', 'RoW']  

In [26]:
# Define the user-defined ranking
if esm_location == 'CA-QC':
    my_ranking = [
        'CA-QC', # Quebec
        'CA', # Canada
        'CAN', # Canada in IMAGE
        'CAZ', # Canada - Australia - New Zealand in REMIND
        'RNA', # North America
        'US', # United States
        'USA', # United States in REMIND and IMAGE
        'GLO', # Global average 
        'RoW', # Rest of the world
    ]
elif esm_location == 'CH':
    my_ranking = [
        'CH',
        'NEU',
        'EUR',
        'WEU',
        'RER', 
        'IAI Area, EU27 & EFTA',
        'GLO',
        'RoW'
    ]
else:
    my_ranking = [
        'GLO',
        'RoW',
    ]

## Create a new database with additional CPC categories (optional)
In case you are working with a LCI database without CPC categories, you can create a new database with the CPC categories. The function `create_new_database_with_CPC_categories` takes as input the database with missing CPC categories, the name of the new database, and a mapping between the products and activities in the LCI database and the CPC categories. It creates a new database with the CPC categories. This step can take a few minutes depending on the size of the database.

In [27]:
# If necessary, add missing CPC categories to the database
premise_db.add_CPC_categories()

## Relink the mapping dataframe with another database (optional)
In this example, we relink the current mapping, which is based on ecoinvent and some additional inventories, with a [premise](https://premise.readthedocs.io/en/latest/introduction.html) database. The function `create_complementary_database` takes as input the mapping dataframe, the database to link to, and the name of a newly created complementary database, in case some LCI datasets are missing in the LCI database we relink to. It returns a new mapping dataframe with the location column updated, and create the complementary database in the brightway project.

In [28]:
name_premise_comp_db = name_premise_db + f'_comp_{esm_location}'

In [29]:
mapping_linked_to_premise = premise_db.create_complementary_database(
    df_mapping=mapping,
    complement_db_name=name_premise_comp_db,
    premise_changes=premise_changes,
)

Getting activity data


100%|██████████| 28471/28471 [00:00<00:00, 32765.08it/s]


Adding exchange data to activities


100%|██████████| 832966/832966 [00:42<00:00, 19524.05it/s]


Filling out exchange data


...

Adding exchange data to activities


100%|██████████| 30702/30702 [00:00<00:00, 43502.03it/s]


Filling out exchange data


100%|██████████| 935/935 [00:00<00:00, 2325.32it/s]


Loaded regional delivery_truck from brightway!
The complementary database did not have to be created


In [30]:
if name_premise_db+f'_comp_{esm_location}' in bd.databases: # if the complementary database exists, load it and update it with CPC categories
    premise_comp_db = Database(db_names=name_premise_db) # complementary database
    premise_comp_db.add_CPC_categories() # if necessary, add CPC categories
    main_db = premise_db + premise_comp_db # concatenate the two databases in one database object (without writing it)
else:
    main_db = premise_db

In [31]:
mapping_linked_to_premise.head()

,Name,Type,Product,Activity,Location,Database
0,AFC,Construction,"fuel cell system, 1 kWe, proton exchange membr...","fuel cell system assembly, 1 kWe, proton excha...",GLO,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
1,AFC,Operation,"electricity, from residential heating system","electricity, residential, by conversion of hyd...",CH,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
2,ALKALINE_ELECTROLYSIS,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
3,ALKALINE_ELECTROLYSIS_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
4,ALKALINE_ELECTROLYSIS_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...


## Initialize the ESM database

In [32]:
esm = ESM(
    # Mandatory inputs
    mapping=mapping_linked_to_premise,
    unit_conversion=unit_conversion,
    model=model,
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC,
    main_database=main_db,
    esm_db_name=esm_db_name,
    
    # Optional inputs
    technology_compositions=technology_compositions,
    tech_specifics=technology_specifics,
    lifetime=lifetime,
    efficiency=efficiency,
    regionalize_foregrounds=regionalize_foregrounds,
    accepted_locations=accepted_locations,
    locations_ranking=my_ranking,
    esm_location=esm_location,
    spatialized_database=spatialized_database,
    spatialized_biosphere_db=spatialized_biosphere_db,
    results_path_file=f'results/energyscope_{esm_location}/{esm_year}/',
)

In [33]:
esm.check_inputs()

List of technologies or resources that are in the model file but not in the mapping file. Their impact scores will be set to the default value.

--> ['CO2_E', 'DEC_RENOVATION', 'DHN_COGEN_WASTE_CC', 'DHN_RENOVATION', 'DIESEL_S', 'ELEC_S', 'GASOLINE_S', 'H2_NG_CCGT_CCS', 'HT_LT', 'IND_BOILER_WASTE_CC', 'IND_COGEN_WASTE_CC', 'LT_DEC_WH', 'LT_DHN_WH', 'NG_S', 'RES_GEO', 'RES_HYDRO', 'RES_SOLAR', 'RES_TIDAL', 'RES_WIND_OFFSHORE', 'RES_WIND_ONSHORE', 'STO_CO2', 'STO_DIE', 'STO_ELEC', 'STO_GASO', 'STO_H2', 'STO_NG']

List of technologies or resources that are in the mapping file but not in the model file (this will not be a problem in the workflow).

--> ['BATTERY', 'DEC_TH_STORAGE', 'DHN_TH_STORAGE', 'EHV_GRID', 'HV_GRID', 'HYDRO_STORAGE', 'LV_GRID', 'MV_GRID']

List of technologies that are in the efficiency file but not in the mapping file (this will not be a problem in the workflow).

--> ['H2_NG_CCGT_CCS', 'IND_BOILER_WASTE_CC']

List of technologies that are in the tech_specifics file 

## Add or replace the location column based on a user-defined ranking (optional)
Based on a user-defined ranking, the location column of the mapping dataframe can be updated. The function `change_location_mapping_file` takes as input the mapping dataframe, the user-defined ranking, and the base database. It returns the mapping dataframe with the location column updated.

In [34]:
# Update mapping dataframe with better locations
esm.change_location_mapping_file()

In [35]:
esm.mapping.head() 

,Name,Type,Product,Activity,Location,Database
0,AFC,Construction,"fuel cell system, 1 kWe, proton exchange membr...","fuel cell system assembly, 1 kWe, proton excha...",GLO,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
1,AFC,Operation,"electricity, from residential heating system","electricity, residential, by conversion of hyd...",CH,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
2,ALKALINE_ELECTROLYSIS,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CAN,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
3,ALKALINE_ELECTROLYSIS_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
4,ALKALINE_ELECTROLYSIS_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...


In [36]:
# Save the relinked mapping file for later use
esm.mapping.to_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}_linked.csv', index=False)

## Create ESM database after double-counting removal, efficiency harmonization, and lifetime harmonization

In [37]:
# Load the relinked mapping file to skip the previous steps
# mapping_linked_to_premise = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}_linked.csv')
# esm.mapping = mapping_linked_to_premise

In [38]:
esm.create_esm_database()

### Starting to add construction and resource activities database ###
No location found in your ranking for waste polystyrene - market group for waste polystyrene
--> Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste polyvinylchloride - market group for waste polyvinylchloride
--> Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste plastic, mixture - market group for waste plastic, mixture
--> Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste polyethylene - market group for waste polyethylene
--> Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste glass - market group for waste glass
--> Have to keep the initial location: RER
No location found in your ranking for iridium - platinum group metal, extraction and refinery operations
--> Have to keep the initial location: ZA

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 10/09/2024 11:23:21
  Finished: 10/09/2024 11:23:22
  Total time elapsed: 00:00:01
  CPU %: 3.20
  Memory %: 8.07
### Database written ###
--> Time: 369.0 seconds


In [39]:
# Save the mapping file with new codes for later use
esm.mapping.to_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}_new_code.csv', index=False)

## Computing the LCA metrics 

In [40]:
# To skip the previous steps (if the mapping file with the new codes has already been created)
# mapping_linked_to_premise_new_code = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}_new_code.csv')
# esm.mapping = mapping_linked_to_premise_new_code

In [41]:
R_long = esm.compute_impact_scores(methods=lcia_methods)

Getting activity data


100%|██████████| 1159/1159 [00:00<00:00, 100201.97it/s]


Adding exchange data to activities


100%|██████████| 34255/34255 [00:01<00:00, 29404.39it/s]


Filling out exchange data


100%|██████████| 1159/1159 [00:04<00:00, 265.78it/s]


Loaded EnergyScope_CA-QC_2020 from brightway!


In [42]:
R_long.Value *= 1e6 # from FU / kW(h)-pkm-tkm to FU / GW(h)-Mpkm-Mtkm

In [43]:
R_long.head()

,Impact_category,New_code,Value,Name,Type
0,"(IMPACT World+ Damage 2.0.1_regionalized, Ecos...",0lu96hf3pt4h1xmy8r4lpmzdnwu3bhwe,3.642961e+09,AFC,Construction
1,"(IMPACT World+ Damage 2.0.1_regionalized, Ecos...",0lu96hf3pt4h1xmy8r4lpmzdnwu3bhwe,1.171020e+09,AFC,Construction
2,"(IMPACT World+ Damage 2.0.1_regionalized, Huma...",0lu96hf3pt4h1xmy8r4lpmzdnwu3bhwe,1.658111e+04,AFC,Construction
3,"(IMPACT World+ Damage 2.0.1_regionalized, Huma...",0lu96hf3pt4h1xmy8r4lpmzdnwu3bhwe,5.411366e+03,AFC,Construction
4,"(IMPACT World+ Midpoint 2.0.1_regionalized, Mi...",0lu96hf3pt4h1xmy8r4lpmzdnwu3bhwe,6.206170e+09,AFC,Construction


In [44]:
R_long.to_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores.csv', index=False)

## Convert the results in an AMPL format

In [45]:
# To skip the previous steps
# R_long = pd.read_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores.csv')

In [46]:
lcia_method = 'IMPACT World+ Damage 2.0.1_regionalized - Total only'

In [47]:
# Additional information that can be added at the beginning of the AMPL .mod and .dat files
metadata = {
    'ecoinvent_version': ecoinvent_version,
    'year': esm_year,
    'spatialized': spatialized_database,
    'regionalized': regionalize_foregrounds,
    'iam': premise_iam,
    'ssp_rcp': premise_ssp_rcp,
    'lcia_method': lcia_method,
}

### Normalize LCA indicators and create the .dat file

In [48]:
esm.normalize_lca_metrics(
    R=R_long,
    mip_gap=1e-6,
    lcia_method=lcia_method,
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/{esm_year}/',
    metadata=metadata,
)

### Create the .mod file

In [49]:
esm.generate_mod_file_ampl(
    lcia_method=lcia_method,
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/{esm_year}/',
    metadata=metadata,
)

## Integrate the ESM results back in the LCI database

In [50]:
esm.create_new_database_with_esm_results(
    esm_results=results_from_esm,
    new_end_use_types=new_end_use_types,
    tech_to_remove_layers=technologies_to_remove_from_layers,
    write_database=False,
)

No location found in your ranking for hard coal - hard coal, import from RNA
--> Have to keep the initial location: IN
No location found in your ranking for hard coal - hard coal, import from ZA
--> Have to keep the initial location: IN
No location found in your ranking for hard coal - hard coal, import from RU
--> Have to keep the initial location: IN
No location found in your ranking for ethanol, from eucalyptus - ethanol production, via fermentation, from eucalyptus
--> Have to keep the initial location: INDIA
The technology DHN_RENOVATION is not in the mapping file. It cannot be considered in the result LCI dataset.
No location found in your ranking for kerosene - kerosene, import from RoW
--> Have to keep the initial location: ZA
No location found in your ranking for natural gas, high pressure - evaporation of natural gas
--> Have to keep the initial location: JP
